# 🍎 Chat Completions with AIProjectClient 🍏

In this notebook, we'll demonstrate how to perform **Chat Completions** using the **Azure AI Foundry** SDK. We'll combine **`azure-ai-projects`** and **`azure-ai-inference`** packages to:

1. **Initialize** an `AIProjectClient`.
2. **Obtain** a Chat Completions client to do direct LLM calls.
3. **Use** a **prompt template** to add system context.
4. **Send** user prompts in a health & fitness theme.

## 🏋️ Health-Fitness Disclaimer
> **This example is for demonstration only and does not provide real medical advice.** Always consult a professional for health or medical-related questions.

### Prerequisites
Before starting this notebook, please ensure you have completed all prerequisites listed in the root [README.md](../../README.md#-prerequisites).

Let's get started! 🎉

<img src="./seq-diagrams/1-chat.png" width="30%"/>


## 1. Initial Setup
Load environment variables, create an `AIProjectClient`, and fetch a `ChatCompletionsClient`. We'll also define a **prompt template** to show how you might structure a system message.


In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
from azure.identity import DefaultAzureCredential

from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, SystemMessage  # for chat messages

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent.parent
load_dotenv(parent_dir / '.env')

# Retrieve from environment
connection_string = os.environ.get("PROJECT_CONNECTION_STRING")
model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME")

try:
    # Create the project client
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=connection_string,
    )
    print("✅ Successfully created AIProjectClient")
except Exception as e:
    print("❌ Error initializing client:", e)


✅ Successfully created AIProjectClient


### Prompt Template
We'll define a quick **system** message that sets the context as a friendly, disclaimer-providing fitness assistant.

```txt
SYSTEM PROMPT (template):
You are FitChat GPT, a helpful fitness assistant.
Always remind users: I'm not a medical professional.
Be friendly, provide general advice.
...
```

We'll then pass user content as a **user** message.


In [2]:
# We'll define a function that runs chat completions with a system prompt & user prompt
def chat_with_fitness_assistant(user_input: str):
    """Use chat completions to get a response from our LLM, with system instructions."""
    # Our system message template
    system_text = (
        "You are FitChat GPT, a friendly fitness assistant.\n"
        "Always remind users: I'm not a medical professional.\n"
        "Answer with empathy and disclaimers."
    )

    # We'll open the chat completions client
    with project_client.inference.get_chat_completions_client() as chat_client:
        # Construct messages: system + user
        system_message = SystemMessage(content=system_text)
        user_message = UserMessage(content=user_input)

        # Send the request
        response = chat_client.complete(
            model=model_deployment,
            messages=[system_message, user_message]
        )

        return response.choices[0].message.content  # simplest approach: get top choice's content

print("Defined a helper function to do chat completions.")

Defined a helper function to do chat completions.


## 2. Try Chat Completions 🎉
We'll call the function with a user question about health or fitness, and see the result. Feel free to modify the question or run multiple times!


In [3]:
user_question = "How can I start a beginner workout routine at home?"
reply = chat_with_fitness_assistant(user_question)
print("🗣️ User:", user_question)
print("🤖 Assistant:", reply)

🗣️ User: How can I start a beginner workout routine at home?
🤖 Assistant: That’s a great question! First, I’m not a medical professional, so be sure to check with your doctor before starting any new exercise program, especially if you have any health concerns or conditions.

Getting started with a beginner workout at home is a fantastic decision! Here’s a simple, balanced routine you can try. All you need is comfortable clothing, some space, and maybe a mat or towel for floor exercises.

**Sample Beginner Home Workout:**

1. **Warm-Up (5 minutes)**
   - March in place
   - Arm circles
   - Gentle jumping jacks or stepping side-to-side

2. **Circuit (Repeat 2-3 times):**
   - **Bodyweight Squats:** 10-15 reps  
     Stand with feet shoulder-width apart, squat down as if sitting in a chair, then stand back up.

   - **Push-Ups (regular or knee):** 5-10 reps  
     Modify by doing them on your knees if needed.

   - **Glute Bridges:** 10-15 reps  
     Lie on your back, feet flat on the f

## 3. Another Example: Prompt Template with Fill-Ins 📝
We can go a bit further and add placeholders in the system message. For instance, imagine we have a **userName** or **goal**. We'll show a minimal example.


In [4]:
def chat_with_template(user_input: str, user_name: str, goal: str):
    # Construct a system template with placeholders
    system_template = (
        "You are FitChat GPT, an AI personal trainer for {name}.\n"
        "Your user wants to achieve: {goal}.\n"
        "Remind them you're not a medical professional. Offer friendly advice."
    )

    # Fill in placeholders
    system_prompt = system_template.format(name=user_name, goal=goal)

    with project_client.inference.get_chat_completions_client() as chat_client:
        system_msg = SystemMessage(content=system_prompt)
        user_msg = UserMessage(content=user_input)

        response = chat_client.complete(
            model=model_deployment,
            messages=[system_msg, user_msg]
        )

    return response.choices[0].message.content

# Let's try it out
templated_user_input = "What kind of home exercise do you recommend for a busy schedule?"
assistant_reply = chat_with_template(
    templated_user_input,
    user_name="Jordan",
    goal="increase muscle tone and endurance"
)
print("🗣️ User:", templated_user_input)
print("🤖 Assistant:", assistant_reply)

🗣️ User: What kind of home exercise do you recommend for a busy schedule?
🤖 Assistant: Absolutely! I’m glad you’re looking to fit exercise into your busy schedule. Remember, I’m not a medical professional, so please check with your healthcare provider if you have any concerns or medical conditions.

For increasing muscle tone and endurance at home, efficiency is key. Here are some recommendations:

**1. High-Intensity Interval Training (HIIT):**   
Short, intense bursts of exercise followed by brief rest. 20-30 minutes is plenty! Example:  
- 30 seconds squats  
- 15 seconds rest  
- 30 seconds push-ups  
- 15 seconds rest  
- 30 seconds jumping jacks  
- 15 seconds rest  
- Repeat 3-4 times

**2. Bodyweight Strength Circuits:**   
Choose 4-6 exercises: squats, lunges, planks, push-ups, dips (using a chair), and burpees. Perform each for 30-45 seconds, with minimal rest between moves. Do 2-4 rounds.

**3. Resistance Bands:**  
These are great for a quick workout. You can do rows, chest

## 🎉 Congratulations!
You've successfully performed **chat completions** with the Azure AI Foundry's `AIProjectClient` and `azure-ai-inference`. You've also seen how to incorporate **prompt templates** to tailor your system instructions.

#### Head to [2-embeddings.ipynb](2-embeddings.ipynb) for the next part of the workshop! 🎯